## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,WeatherPy_Database.csv,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Khasan,RU,42.4308,130.6434,50.23,96,100,19.77,overcast clouds
1,1,Dzhebariki-Khaya,RU,62.2167,135.8000,70.74,51,19,2.01,few clouds
2,2,Hilo,US,19.7297,-155.0900,89.28,87,100,10.36,overcast clouds
3,3,Bluff,NZ,-46.6000,168.3333,46.80,66,100,11.97,overcast clouds
4,4,Zhigansk,RU,66.7697,123.3711,59.09,56,48,9.26,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,WeatherPy_Database.csv,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Hilo,US,19.7297,-155.0900,89.28,87,100,10.36,overcast clouds
7,7,Port-Gentil,GA,-0.7193,8.7815,75.16,94,20,2.30,few clouds
18,18,Vaini,TO,-21.2000,-175.2000,76.26,78,19,11.54,few clouds
23,23,Constantine,DZ,36.3650,6.6147,76.41,60,0,1.14,clear sky
25,25,San Cristobal,VE,7.7669,-72.2250,78.98,94,85,4.79,overcast clouds
30,30,Nortelandia,BR,-14.4547,-56.8028,78.13,47,28,1.77,scattered clouds
32,32,Baracoa,CU,20.3467,-74.4958,85.87,58,99,8.81,overcast clouds
33,33,Atuona,PF,-9.8000,-139.0333,78.31,74,32,15.17,scattered clouds
37,37,Sinnamary,GF,5.3833,-52.9500,78.19,87,94,9.26,overcast clouds
49,49,Kloulklubed,PW,7.0419,134.2556,75.70,70,41,5.19,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

WeatherPy_Database.csv    203
City                      203
Country                   203
Lat                       203
Lng                       203
Max Temp                  203
Humidity                  203
Cloudiness                203
Wind Speed                203
Current Description       203
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

WeatherPy_Database.csv    203
City                      203
Country                   203
Lat                       203
Lng                       203
Max Temp                  203
Humidity                  203
Cloudiness                203
Wind Speed                203
Current Description       203
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hilo,US,89.28,overcast clouds,19.7297,-155.0900,
7,Port-Gentil,GA,75.16,few clouds,-0.7193,8.7815,
18,Vaini,TO,76.26,few clouds,-21.2000,-175.2000,
23,Constantine,DZ,76.41,clear sky,36.3650,6.6147,
25,San Cristobal,VE,78.98,overcast clouds,7.7669,-72.2250,
30,Nortelandia,BR,78.13,scattered clouds,-14.4547,-56.8028,
32,Baracoa,CU,85.87,overcast clouds,20.3467,-74.4958,
33,Atuona,PF,78.31,scattered clouds,-9.8000,-139.0333,
37,Sinnamary,GF,78.19,overcast clouds,5.3833,-52.9500,
49,Kloulklubed,PW,75.70,scattered clouds,7.0419,134.2556,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df = hotel_df.replace("", nan_value)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Hilo,US,89.28,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
7,Port-Gentil,GA,75.16,few clouds,-0.7193,8.7815,Fengshui Residence
18,Vaini,TO,76.26,few clouds,-21.2000,-175.2000,Keleti Beach Resort
23,Constantine,DZ,76.41,clear sky,36.3650,6.6147,Grand Hotel Cirta
25,San Cristobal,VE,78.98,overcast clouds,7.7669,-72.2250,Pirineos
...,...,...,...,...,...,...,...
674,Ca Mau,VN,76.82,overcast clouds,9.1769,105.1500,Mường Thanh Luxury Cà Mau Hotel
675,San Francisco,US,75.45,overcast clouds,37.7749,-122.4194,Edwardian Hotel
679,Pangai,TO,77.79,clear sky,-19.8000,-174.3500,Ha'apai Beach Resort
681,Qui Nhon,VN,79.14,few clouds,13.7667,109.2333,Khách sạn Hải Âu Quy Nhơn - Seagull Hotel


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))